In [2]:
!pip install faiss-gpu-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 MB 18.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 6.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 88.8 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
import faiss
import torch
from tqdm.auto import trange
import pickle
import re
import torch.functional as F

2024-10-17 16:49:31.103030: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 16:49:31.103086: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 16:49:31.104214: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-17 16:49:31.110744: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-17 16:49:32.037875: W tensorflow/compiler/tf2

In [2]:
def parse_dialog(sentences):
    return '; '.join(sentences)

def prepare_text(s):
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

def parse_request(x):
    return f"{x['dialog']} {x['question']} {x['choice_0']} {x['choice_1']}  {x['choice_2']}"

def make_df(path, is_test=False):
    data = pd.read_json(path,lines=True)
    df = []
    
    for i in range(len(data)):
        row = data.iloc[i]
        dialog = parse_dialog(row[0])
        for ques in row[1]:
            df.append({
                'dialog_id': i,
                'dialog': dialog,
                'question': ques['question'],
                'choice_0': ques['choice'][0],
                'choice_1': ques['choice'][1],
                'choice_2': ques['choice'][2],
                'labels': -100 if is_test else ques['choice'].index(ques['answer'])
            })
    return pd.DataFrame(df)

In [3]:
train_data = make_df('data/train.statement.jsonl')
vall_data = make_df('data/dev.statement.jsonl')
test_data = make_df('data/test.statement.jsonl',is_test=True)

In [4]:
train_data['request'] = train_data.apply(parse_request,axis=1)
vall_data['request'] = vall_data.apply(parse_request,axis=1)
test_data['request'] = test_data.apply(parse_request,axis=1)

In [5]:
encoder = SentenceTransformer('dunzhang/stella_en_1.5B_v5', trust_remote_code=True).cuda()

In [6]:
@torch.no_grad()
def create_embeds(texts, model, query_prompt=None,prompt_name = "s2p_query",batch_size=64, max_length=1024):
    model.max_seq_length = max_length
    model.tokenizer.model_max_length = max_length
    if query_prompt:
        texts = [query_prompt + x for x in texts]
    
    embeds = model.encode(
        texts,
        batch_size=batch_size,
        prompt_name=prompt_name,
        return_tensors=True,
        normalize_embeddings=True,
        show_progress_bar=True
    )
    return np.stack(embeds,dtype=np.float32)

In [7]:
graph_state = pickle.load(open('graph.pkl','rb'))

In [8]:
concepts = [prepare_text(s)  if type(s) == str else ' ' for s in graph_state['concepts_maper']]

In [9]:
features = create_embeds(
    concepts,
    encoder,
    batch_size=64,
    prompt_name=None,
    max_length=64,
    query_prompt="Instruct: Given a small concept query, retrieve relevant texts in which this concept is contained.\nQuery: "
)

Batches:   0%|          | 0/12489 [00:00<?, ?it/s]

In [10]:
features.shape

(799272, 1024)

In [27]:
index = faiss.IndexFlatIP(features.shape[1]) #use faiss.IndexFlatL2 for l2 distance
faiss.normalize_L2(features)
index.add(features)
res = faiss.StandardGpuResources()
index = faiss.index_cpu_to_gpu(res, 0, index)
index.ntotal

799272

In [32]:
train_features = create_embeds(train_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)

  0%|          | 0/58 [00:00<?, ?it/s]

In [33]:
test_features = create_embeds(test_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)

  0%|          | 0/17 [00:00<?, ?it/s]

In [36]:
val_features = create_embeds(vall_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)

  0%|          | 0/7 [00:00<?, ?it/s]

In [37]:
faiss.normalize_L2(train_features)
D_train,I_train = index.search(train_features,k=25)

In [38]:
faiss.normalize_L2(test_features)
D_test,I_test = index.search(test_features,k=25)

In [39]:
faiss.normalize_L2(test_features)
D_vall,I_vall = index.search(val_features,k=25)

In [43]:
train_data['candidates'] = [[concepts[j] for j in x] for x in I_train]
test_data['candidates'] = [[concepts[j] for j in x] for x in I_test]
vall_data['candidates'] = [[concepts[j] for j in x] for x in I_vall]

In [48]:
train_data[['request','candidates']].sample(1,random_state=56).values

array([["W: Your boarding pass, please!; M: Here it is.; W: Thanks.; M: Where is my seat?; W: 15A. Just over there, sir.; M: Thank you, Miss. By the way, where can I put my bag'?; W: You can put your coat and small things on the shelf over your feet.; M: I've got two bags. I'll afraid I can't put them down there. Can I put them on the shelf above?; W: All right, sir.; M: And where is the washroom?; W: Behind the curtain.; M: Thank you again.; W: You're welcome. What is the relationship between the two speakers? Conductor and driver Air hostess and passenger.  Waitress and customer.",
        list(['talk to passenger', 'may talk to hostess', 'talking with hostess', 'passengers to sit in', 'elevator talk', 'guests of airline', 'talking in', 'job conversation', 'carry on conversation', 'first line of conversation', 'passenger to sit on', 'talking to customer', 'open ended conversation', 'discussing available seating with customers', 'seating passengers', 'speaking to receptionist', 'board